# **Earth Mover's Distance (1st Wasserstein Distance)**

---
## Definition:


![替代文字](https://wikimedia.org/api/rest_v1/media/math/render/svg/29baf892051625c69a9c3ba2f0604f0fbf153330)

where **p=1** (from wikipedia->[Wasserstein metric](https://en.wikipedia.org/wiki/Wasserstein_metric)).

And it can be calculated by $\int^{+\infty}_{-\infty}|U-V|$  as show in Scipy->[wasserstein_distance](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.wasserstein_distance.html).

## Description:

In statistics, the earth mover's distance (EMD) is a measure of the distance between **two probability distributions** over a region D. In mathematics, this is known as the Wasserstein metric. 

Informally, if the distributions are interpreted as two different ways of piling up a certain amount of dirt over the region D, the EMD is the **minimum cost** of turning one pile into the other; where the cost is assumed to be amount of dirt moved times the distance by which it is moved.

## Usage:

Wassertein GAN (WGAN)

# Example

scipy.stats.wasserstein_distance(u_values, v_values, u_weights=None, v_weights=None)

NOTE: This func actually calculate the **1st** wasserstein distance

In [1]:
# The example of wassertein_distance func

import numpy as np
from scipy.stats import wasserstein_distance

u_values = np.array([0,1,3], dtype=np.uint8)
v_values = np.array([5,6,8], dtype=np.uint8)
c=2
# The minimum cost of turning left distribution (observations) to right distribution (observations),
# ex. 0->5, 1->6, 3->8
print("W(u,v):", wasserstein_distance(u_values, v_values))
print("W(v,u):", wasserstein_distance(v_values, u_values))
print("W(u,v+c):", wasserstein_distance(u_values, v_values+c))

W(u,v): 5.0
W(v,u): 5.0
W(u,v+c): 7.0


The wasserstein_distance() call _cdf_distance(p, u_values, v_values, u_weights=None, v_weights=None) with p=1. 

The scipy's implementation is calculate $\int^{+\infty}_{-\infty}|U-V|$, which is the integral of |u's CDF- v's CDF| 

The calculation details are as follows $\downarrow$

In [2]:
# Get the sorter of u and v
u_sorter = np.argsort(u_values)
v_sorter = np.argsort(v_values)

# Concatenate the u and v (Two 1-D ndarray)
# NOTE: np.concatenate() does not increse the dimension
all_values = np.concatenate((u_values, v_values)) # ->[0,1,3,5,6,8]
all_values.sort(kind='mergesort') # sort all_values in-place, ->[0,1,3,5,6,8]

# Compute the differences between pairs of successive values of u and v.
deltas = np.diff(all_values) # ->[1,2,2,1,2]

# Get the respective positions of the values of u and v among the values of
# both distributions.
u_cdf_indices = u_values[u_sorter].searchsorted(all_values[:-1], 'right') 
v_cdf_indices = v_values[v_sorter].searchsorted(all_values[:-1], 'right')
# u_cdf_indices ->[1,2,3,3,3], v_cdf_indices ->[0,0,0,1,2]

# Calculate the CDFs of u and v
u_cdf = u_cdf_indices / u_values.size
v_cdf = v_cdf_indices / v_values.size
# u_cdf ->[1/3,2/3,1,1,1], v_cdf ->[0,0,0,1/3,2/3]

# Compute the value of the integral based on the CDFs.
emd = np.sum(np.multiply(np.abs(u_cdf - v_cdf), deltas))

print(emd)

5.0
